# Extracting Data of 2000s

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

# Fetch the website content
base_url = 'https://www.giitaayan.com/search.asp?PageNo={}&s=200&browse=keywords'

#build the big dataframe of all records
total_data = pd.DataFrame()

for i in range(1,28):
   url = base_url.format(i)
   response = requests.get(url)
   data = pd.read_html(url)
   data1 = data[1]
   total_data = pd.concat([total_data,data1],ignore_index = True)




In [4]:
total_data.head()

,0,1,2,3,4
0,Song,Film,Music Director,Lyricist,Singers
1,"##Ave Maria, gracia plena## - View",Sur (2002),M M Kreem,Nida Fazli,Marijke Desouza
2,##baby baby## ... shaa naa naa ... sun baabaa ...,Krodh/ Blast of Anger (2000),Anand-Milind,Deepak Choudhary,"Vinod Rathod, Sapana Mukherji,..."
3,##boss## kaun thaa maaluum hai kyaa - View,Jhankaar Beats (2003),Vishal-Shekhar,Vishal Dadlani,"Amit Kumar, Vishal Dadlani, Sh..."
4,##censor## - View,Censor (2001),Jatin-Lalit,Vinu Mahendra,"Vinod Rathod, Jaspindar Narula"


In [5]:
total_data.columns = ["Song","Film","Music Director","Lyricist","Singers"]

In [6]:
total_data = total_data.drop(0,axis=0)

In [7]:
total_data['Song'] = total_data['Song'].str.replace("View","")
total_data['Song'] = total_data['Song'].str.replace("##","")
total_data['Song'] = total_data['Song'].str.replace("-","")

In [259]:
total_data.to_excel("Whole_data_2000s.xlsx")

In [8]:
data2 = total_data.groupby(['Music Director']).size().reset_index(name='Count_Song')

In [30]:
data3 = data2.sort_values(by="Count_Song",ascending=False)
data3.reset_index(inplace=True)

In [31]:
data3 = data3.drop('index',axis=1)
data3 = data3.drop(0,axis=0)

In [32]:
data3['MusicDirectorID'] = range(1, len(data3) + 1)


In [33]:
data3

,Music Director,Count_Song,MusicDirectorID
1,Jatin-Lalit,95,1
2,A R Rahman,91,2
3,Nadeem-Shravan,88,3
4,Anand Raj Anand,74,4
5,Shankar-Ehsaan-Loy,60,5
...,...,...,...
109,Kalyan Vardhan,1,109
110,Kailash-Naresh-Paresh,1,110
111,Jaidev Kumar,1,111
112,Iqbal Qureshi,1,112


In [11]:
data2_2 = total_data.groupby(['Singers']).size().reset_index(name='Count_Song')

In [35]:
data2_2
data3_3 = data2_2.sort_values(by="Count_Song",ascending=False)
data3_3.reset_index(inplace=True)

In [36]:
data3_3 = data3_3.drop('index',axis=1)


In [37]:
data3_3 = data3_3.drop(0,axis=0)
data3_3['SingerID'] = range(1, len(data3_3) + 1)

In [38]:
data3_3

,Singers,Count_Song,SingerID
1,"Kumar Sanu, Alka Yagnik",32,1
2,Sonu Nigam,31,2
3,"Sonu Nigam, Alka Yagnik",28,3
4,Singers,26,4
5,"Sonu Nigam, Chorus",23,5
...,...,...,...
737,Krishna,1,737
738,"Kumar Sanu, Alisha Chinai",1,738
739,"Kumar Sanu, Alka Yagnik, Ajay ...",1,739
740,"Kumar Sanu, Alka Yagnik, Ram S...",1,740


In [15]:
data4 = total_data.groupby(['Lyricist']).size().reset_index(name='Count_Song')

In [39]:
data4
data4_4 = data4.sort_values(by="Count_Song",ascending=False)
data4_4.reset_index(inplace=True)

In [40]:
data4_4 = data4_4.drop('index',axis=1)
data4_4 = data4_4.drop(0,axis=0)
data4_4['LyricistID'] = range(1, len(data4_4) + 1)

In [41]:
data4_4.head(20)

,Lyricist,Count_Song,LyricistID
1,Javed Akhtar,111,1
2,Anand Bakshi,110,2
3,Gulzar,70,3
4,Dev Kohli,42,4
5,Faiz Anwar,38,5
6,Sudhakar Sharma,36,6
7,Lyricist,26,7
8,Praveen Bhardwaj,24,8
9,Mehboob,24,9
10,Nida Fazli,21,10


In [42]:
bestof = pd.concat([data3.head(20),data3_3.head(20),data4_4.head(20)],axis=1)

In [43]:
bestof.to_excel("Top_20s.xlsx")

# Building Relationships

In [25]:
# Merge data to get the IDs as foreign keys in total_data
filtered_data = total_data[
    (total_data['Music Director'].isin(data3['Music Director'].head(20))) &
    (total_data['Singers'].isin(data3_3['Singers'].head(20))) &
    (total_data['Lyricist'].isin(data4_4['Lyricist'].head(20)))
]

# Merge to add foreign keys
filtered_data = filtered_data.merge(data3[['Music Director', 'MusicDirectorID']], on='Music Director', how='left')
filtered_data = filtered_data.merge(data3_3[['Singers', 'SingerID']], on='Singers', how='left')
filtered_data = filtered_data.merge(data4_4[['Lyricist', 'LyricistID']], on='Lyricist', how='left')

# Select relevant columns for the song master table
song_master_table = filtered_data[['Song', 'Film', 'MusicDirectorID', 'SingerID', 'LyricistID']].drop_duplicates()


In [26]:
song_master_table = song_master_table.drop("Film",axis=1)

In [28]:
song_master_table.to_excel("relationship.xlsx")